In [ ]:
import os
import pickle
import glob

import pandas as pd 
import numpy as np

bq_in_dir="/home/jupyter/data_transfer/data/"
sim_rec_df = pd.read_csv(os.path.join(bq_in_dir, "hansi_rec_ClicksData_5core.csv"), index_col=0)
compl_rec_df = pd.read_csv(os.path.join(bq_in_dir, "comp_rec_ClicksData_2core.csv"), index_col=0)
search_df = pd.read_csv(os.path.join(bq_in_dir, "search_ClicksData_1year_5core.csv"), index_col=0)
product_df = pd.read_csv(os.path.join(bq_in_dir, "all_products_info.csv"), index_col=0)

all_products = set(product_df.product_id)
anchors = set(compl_rec_df.anchor)
compl_ivms = set(compl_rec_df.ivm)
all_compl_ivms = anchors.union(compl_ivms)

print("================================ For anchor_to_compl_ivms: ===================================")
print("number of unique product = {:,}, anchors = {:,}, complementary_compl_ivms = {:,}".format(len(all_products), len(anchors), len(compl_ivms)))
assert len(all_products & anchors) == len(anchors) and len(all_products & compl_ivms) == len(compl_ivms),(
    len(all_products & anchors), len(anchors), len(all_products & compl_ivms), len(compl_ivms)
)

all_sim_ivms = set(sim_rec_df.anchor).union(set(sim_rec_df.ivm))
print("================================ After updating anchor_to_similar_ivms: ===================================")
print("all_compl_ivms = {:,}, all_sim_ivms = {:,}".format(len(all_compl_ivms), len(all_sim_ivms)))
print("sim_compl_intersect = {:,} ({:.3f})".format(len(all_compl_ivms & all_sim_ivms), len(all_compl_ivms & all_sim_ivms) / len(all_compl_ivms)))
print("all_ivms = {:,}".format(len(all_compl_ivms | all_sim_ivms)))
all_ivms = all_compl_ivms | all_sim_ivms

assert len(all_products & all_ivms) == len(all_ivms), (len(all_products & all_ivms), len(all_ivms))

query_to_ivms = search_df.groupby("query")["ivm"].apply(list)
ivm_to_queries = search_df.groupby("ivm")["query"].apply(list)
query_lengths = np.array([len(x) for x in ivm_to_queries.values])
all_queries = set(search_df["query"])
print("all queries = {:,}".format(len(all_queries)))
assert len(all_queries) == len(query_to_ivms), len(query_to_ivms)
print("total ivms (queries) = {:,}, length >=3 = {:,}, length >= 5 = {:,}".format(
    len(query_lengths), np.sum(query_lengths >=3), np.sum(query_lengths >= 5) ))

anchor_to_compl_ivms = compl_rec_df.groupby("anchor")["ivm"].apply(list)
compl_ivms_length = np.array([len(x) for x in anchor_to_compl_ivms.values])
print("================================ For anchor_to_compl_ivms: ===================================")
print("total_compl_ivms = {:,}, length >=3 = {:,}, length >= 5 = {:,}".format(len(compl_ivms_length), np.sum(compl_ivms_length >=3), np.sum(compl_ivms_length >= 5) ))

anchor_to_sim_ivms = sim_rec_df.groupby("anchor")["ivm"].apply(list)


In [5]:
compl_rec_df

,anchor,ivm
0,267048-70-45619,43344-70-17
1,149825-1147-EU5,45575-0-EG5
2,773301-215-003168,4934-215-NA
3,1944512-72913-EPB44PHDG,6483-215-NA
4,191956-37672-260272,6483-215-NA
...,...,...
329987,4767637-350-300BRNL SQT 514 RCS,3672733-350-816 SQT 514 SMT RCAL RCS RBP KC IN...
329988,1798630-350-730HFL SQT 514 CP,3672733-350-816 SQT 514 SMT RCAL RCS RBP KC IN...
329989,804702-350-730HFL SQT 514 6AL,3672733-350-816 SQT 514 SMT RCAL RCS RBP KC IN...
329990,4767626-350-818PGHXPSK RDT 514,3672733-350-816 SQT 514 SMT RCAL RCS RBP KC IN...


In [5]:
import pickle

user_dir = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_user/"

with open(os.path.join(user_dir, "ivm_to_pid.pkl"), "rb") as fin:
    ivm_to_pid = pickle.load(fin)
with open(os.path.join(user_dir, "query_to_qid.pkl"), "rb") as fin:
    query_to_qid = pickle.load(fin)


pid_to_qids = {ivm_to_pid[ivm]: [query_to_qid[query] for query in queries] for ivm, queries in ivm_to_queries.items()}
qid_to_pids = {query_to_qid[query]: [ivm_to_pid[ivm] for ivm in ivms] for query, ivms in query_to_ivms.items()}
aid_to_complpids = {ivm_to_pid[anchor]: [ivm_to_pid[prod] for prod in products] for anchor, products in anchor_to_compl_ivms.items()}
aid_to_simpids = {ivm_to_pid[anchor]: [ivm_to_pid[prod] for prod in products] for anchor, products in anchor_to_sim_ivms.items()}

In [ ]:
# start create graph

import random 
from tqdm import tqdm
import networkx as nx
from collections import defaultdict
random.seed(4680)

test_dir = os.path.join(user_dir, "selected_test_user")
kgc_dir = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/"

test_sim_aids, test_compl_aids, test_search_qids = set(), set(), set()
fn_to_qids = {
    os.path.join(test_dir, "anchors.sim.test.tsv"): test_sim_aids,
    os.path.join(test_dir, "anchors.compl.test.tsv"): test_compl_aids,
    os.path.join(test_dir, "queries.search.test.tsv"): test_search_qids
}
for fn, qids in fn_to_qids.items():
    with open(fn) as fin:
        for line in fin:
            qid, _ = line.strip().split("\t")
            qids.add(int(qid))

train_aid_to_simpids, test_aid_to_simpids = {}, {}
for aid, simpids in tqdm(aid_to_simpids.items(), total=len(aid_to_simpids)):
    if aid in test_sim_aids:
        test_aid_to_simpids[aid] = simpids
    else:
        train_aid_to_simpids[aid] = simpids       

train_aid_to_complpids, test_aid_to_complpids = {}, {}
for aid, complpids in tqdm(aid_to_complpids.items(), total=len(aid_to_complpids)):
    if aid in test_compl_aids:
        test_aid_to_complpids[aid] = complpids
    else:
        train_aid_to_complpids[aid] = complpids
        
train_qid_to_pids, test_qid_to_pids = {}, {}
for qid, pids in tqdm(qid_to_pids.items(), total=len(qid_to_pids)):
    if qid in test_search_qids:
        test_qid_to_pids[qid] = pids
    else:
        train_qid_to_pids[qid] = pids
        
print("number of aid_to_simpids  train = {:,}, test = {:,}".format(len(train_aid_to_simpids), 
                                                                     len(test_aid_to_simpids)))
print("number of aid_to_complpids train = {:,}, test = {:,}".format(len(train_aid_to_complpids), 
                                                                     len(test_aid_to_complpids)))
print("number of qid_to_pids train = {:,}, test = {:,}".format(len(train_qid_to_pids), 
                                                                      len(test_qid_to_pids)))

assert len( set(train_aid_to_simpids.keys()) &  set(test_aid_to_simpids.keys()) ) == 0
assert len( set(train_aid_to_complpids.keys())  & set(test_aid_to_complpids.keys())) == 0
assert len( set(train_qid_to_pids.keys())  & set(test_qid_to_pids.keys())) == 0

G = nx.MultiDiGraph()
SIM_RELATION = "is_similar_to"
COMPL_RELATION = "is_complementary_to"
REL_RELATION = "is_relevant_to"

for aid, sim_pids in train_aid_to_simpids.items():
    triples = [(aid, sim_pid, {"type":SIM_RELATION}) for sim_pid in sim_pids]
    G.add_edges_from(triples)
    
for aid, compl_pids in train_aid_to_complpids.items():
    triples = [(aid, compl_pid, {"type":COMPL_RELATION}) for compl_pid in compl_pids]
    G.add_edges_from(triples)
    
for qid, pids in train_qid_to_pids.items():
    triples = [(pid, qid, {"type": REL_RELATION}) for pid in pids]
    G.add_edges_from(triples)
    
multi_edge_pairs = []
for n, nbrs_dict in tqdm(G.adj.items(), total=G.number_of_nodes()):
    for nbr_node, edge_attrs in nbrs_dict.items():
        assert len(edge_attrs) == 1 or len(edge_attrs) == 2
        if len(edge_attrs) == 2:
            multi_edge_pairs.append((n, nbr_node))
            
print("number of edges = {:,}, number of multi-attr edges = {:,}, ({:.3f})".format(G.number_of_edges(), len(multi_edge_pairs), 
                                                                                   len(multi_edge_pairs)/G.number_of_edges()))

In [10]:
def create_triples(hid, pos_tid, miss_hids, duplicate_pairs, eid_to_text, sampler=None):
    if sampler != None:
        assert type(sampler) == dict, type(sampler)
        if hid not in sampler:
            miss_hids.append(hid)
            return 0
    if eid_to_text[hid] == eid_to_text[pos_tid]:
        duplicate_pairs.append((hid, pos_tid))
        return 0
    
    if sampler != None:
        neg_tid = random.sample(sampler[hid], k=1)[0]
        while neg_tid == pos_tid:
            neg_tid = random.sample(sampler[hid], k=1)[0]
    else:
        neg_tid = random.sample(range(2_000_000), k=1)[0]
        while neg_tid == pos_tid:
            neg_tid = random.sample(range(2_000_000), k=1)[0]
            
    return (hid, pos_tid, neg_tid)


eid_to_text = {}
with open(os.path.join(user_dir, "all_entities.tsv")) as fin:
    for line in fin:
        eid, text = line.strip().split("\t")
        eid_to_text[int(eid)] = text
        
run_path = os.path.join(user_dir, "runs/bm25.all.run")
df = pd.read_csv(run_path, sep=" ", names=["hid", "q0", "tid", "rank", "score", "model_name"])
bm25_hid_to_tids = {}
ignore_hids = set()
for hid, group in df.groupby("hid"):
    cand_tids = list(group.tid.values)
    if len(cand_tids) < 10:
        ignore_hids.add(int(hid))
    else:
        bm25_hid_to_tids[int(hid)] = [int(x) for x in cand_tids]
        
print("number of ignore hids = {}".format(len(ignore_hids)))

number of ignore hids = 6644


In [11]:
max5_h2sp = {}
max5_h2cp = {}
max5_h2q = {}

for head_node, nbrs_dict in tqdm(G.adj.items(), total=G.number_of_nodes()):
    sim_pids = []
    compl_pids = []
    rel_qids = []
    for tail_node, edge_attrs in nbrs_dict.items():
        assert len(edge_attrs) == 1 or len(edge_attrs) == 2
        relations = []
        for no, edge_attr in edge_attrs.items():
            relations.append(edge_attr["type"])
        for rel in relations:
            assert rel in [SIM_RELATION, COMPL_RELATION, REL_RELATION]
            if rel in SIM_RELATION:
                sim_pids.append(tail_node)
            if rel in COMPL_RELATION:
                compl_pids.append(tail_node)
            if rel in REL_RELATION:
                rel_qids.append(tail_node)
    if len(sim_pids) != 0:
        max5_h2sp[head_node] = random.sample(sim_pids, k=len(sim_pids))[:5]
    if len(compl_pids) != 0:
        max5_h2cp[head_node] = random.sample(compl_pids, k=len(compl_pids))[:5]
    if len(rel_qids) != 0:
        max5_h2q[head_node] = random.sample(rel_qids, k=len(rel_qids))[:5]
        
miss_hids = []
duplicate_pairs = []

h2sp_triples = []
h2cp_triples = []
q2h_triples = []
for hid, tail_ids in max5_h2sp.items():
    for pos_tid in tail_ids:
        triple = create_triples(hid, pos_tid, miss_hids, duplicate_pairs, eid_to_text)
        if triple != 0:
            h2sp_triples.append(triple)
print("miss_hids = {:,}, duplicate_pairs = {:,}".format(len(miss_hids), len(duplicate_pairs)))
print("="*75)
for hid, tail_ids in max5_h2cp.items():
    for pos_tid in tail_ids:
        triple = create_triples(hid, pos_tid, miss_hids, duplicate_pairs, eid_to_text, sampler=bm25_hid_to_tids)
        if triple != 0:
            h2cp_triples.append(triple)
print("miss_hids = {:,}, duplicate_pairs = {:,}".format(len(miss_hids), len(duplicate_pairs)))
print("="*75)
for pos_tid, head_ids in max5_h2q.items():
    for hid in head_ids:
        triple = create_triples(hid, pos_tid, miss_hids, duplicate_pairs, eid_to_text, sampler=bm25_hid_to_tids)
        if triple != 0:
            q2h_triples.append(triple)
print("miss_hids = {:,}, duplicate_pairs = {:,}".format(len(miss_hids), len(duplicate_pairs)))
print("="*75)

100%|██████████| 1361782/1361782 [00:26<00:00, 51086.91it/s] 


miss_hids = 0, duplicate_pairs = 32,389
miss_hids = 0, duplicate_pairs = 32,755
miss_hids = 39,026, duplicate_pairs = 32,755


In [12]:
max5_q2p = {}
max2_q2p = {}
max5_q2p_triples = []
max2_q2p_triples = []
for qid, pids in train_qid_to_pids.items():
    max5_q2p[qid] = random.sample(pids, k=len(pids))[:5]
    max2_q2p[qid] = random.sample(pids, k=len(pids))[:2]
    
for qid, pos_pids in max5_q2p.items():
    for pos_pid in pos_pids:
        triple = create_triples(qid, pos_pid, miss_hids, duplicate_pairs, eid_to_text, sampler=bm25_hid_to_tids)
        if triple != 0:
            max5_q2p_triples.append(triple)
print("miss_hids = {:,}, duplicate_pairs = {:,}".format(len(miss_hids), len(duplicate_pairs)))
print("="*75)

for qid, pos_pids in max2_q2p.items():
    for pos_pid in pos_pids:
        triple = create_triples(qid, pos_pid, miss_hids, duplicate_pairs, eid_to_text, sampler=bm25_hid_to_tids)
        if triple != 0:
            max2_q2p_triples.append(triple)
print("miss_hids = {:,}, duplicate_pairs = {:,}".format(len(miss_hids), len(duplicate_pairs)))
print("="*75)

miss_hids = 133,224, duplicate_pairs = 32,755
miss_hids = 210,819, duplicate_pairs = 32,755


In [16]:
print(len(h2sp_triples), len(h2cp_triples), len(q2h_triples), len(max2_q2p_triples))

563262 192324 1017083 1364315


In [19]:
import pickle

# downsample max2_q2p_triples
keep_q2p_num = 1_200_000
all_max2_q2p_triples = max2_q2p_triples
max2_q2p_triples = random.sample(all_max2_q2p_triples, k=keep_q2p_num)


if not os.path.exists(kgc_dir):
    os.mkdir(kgc_dir)
out_dir = os.path.join(kgc_dir, "unified_train/")
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

with open(os.path.join(out_dir, "train_graph.pkl"), "wb") as fout:
    pickle.dump(G, fout)

fn_to_tripleNrel = {
    "a2sp.train.tsv": (h2sp_triples, SIM_RELATION),
    "a2cp.train.tsv": (h2cp_triples, COMPL_RELATION),
    "q2a.train.tsv": (q2h_triples, REL_RELATION),   
    "max2_qorient_q2p.train.tsv": (max2_q2p_triples, REL_RELATION),
}

for fn, (triples, relation) in fn_to_tripleNrel.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for (hid, pos_tid, neg_tid) in triples:
            fout.write(f"{hid}\t{pos_tid}\t{neg_tid}\t{relation}\n")
            
out_dir = os.path.join(kgc_dir, "unified_test/")
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

# for similar items
fn_to_aids = {
    "anchors.train.sim.tsv": list(train_aid_to_simpids.keys()),
    "anchors.test.sim.tsv": list(test_aid_to_simpids.keys()),
}
for fn, aids in fn_to_aids.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for aid in aids:
            text = eid_to_text[aid]
            fout.write(f"{aid}\t{text}\t{SIM_RELATION}\n")
            
fn_to_arels = {
    "arels.train.sim.tsv": [(aid, pid) for aid, simpids in train_aid_to_simpids.items() for pid in simpids],
    "arels.test.sim.tsv": [(aid, pid) for aid, simpids in test_aid_to_simpids.items() for pid in simpids],
}
for fn, arels in fn_to_arels.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for (aid, pid) in arels:
            fout.write(f"{aid}\tQ0\t{pid}\t{1}\n")
            
# for complementary items
fn_to_aids = {
    "anchors.train.compl.tsv": list(train_aid_to_complpids.keys()),
    "anchors.test.compl.tsv": list(test_aid_to_complpids.keys()),
}
for fn, aids in fn_to_aids.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for aid in aids:
            text = eid_to_text[aid]
            fout.write(f"{aid}\t{text}\t{COMPL_RELATION}\n")
fn_to_arels = {
    "arels.train.compl.tsv": [(aid, pid) for aid, complpids in train_aid_to_complpids.items() for pid in complpids],
    "arels.test.compl.tsv": [(aid, pid) for aid, complpids in test_aid_to_complpids.items() for pid in complpids]
}
for fn, arels in fn_to_arels.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for (aid, pid) in arels:
            fout.write(f"{aid}\tQ0\t{pid}\t{1}\n")
            
# for queries
fn_to_qids = {
    "queries.train.tsv": list(train_qid_to_pids.keys()),
    "queries.test.tsv": list(test_qid_to_pids.keys()),
}
for fn, qids in fn_to_qids.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for qid in qids:
            text = eid_to_text[qid]
            fout.write(f"{qid}\t{text}\t{REL_RELATION}\n")
fn_to_qrels = {
    "qrels.train.tsv": [(qid, pid) for qid, pids in train_qid_to_pids.items() for pid in pids],
    "qrels.test.tsv": [(qid, pid) for (qid, pids) in test_qid_to_pids.items() for pid in pids],
}
for fn, qrels in fn_to_qrels.items():
    with open(os.path.join(out_dir, fn), "w") as fout:
        for (qid, pid) in qrels:
            fout.write(f"{qid}\tQ0\t{pid}\t{1}\n")

In [ ]:
# sanity check
out_dir = os.path.join(kgc_dir, "unified_train/")
for path in os.listdir(out_dir):
    path = os.path.join(out_dir, path)
    if "train_graph.pkl" in path:
        continue
    ! wc -l $path
    ! head -n 3 $path
    ! tail -n 3 $path
    print("="*100)

In [29]:
hid, pos_tid, neg_tid = (745636,1173319,629502)
print(hid, eid_to_text[hid])
print(pos_tid, eid_to_text[pos_tid])
print(neg_tid, eid_to_text[neg_tid])


745636 Broan 30-in Ducted Black Undercabinet Range Hood ; Undercabinet Range Hoods
1173319 Broan Wall Cap (Black) ; Range Hood Parts
629502 Broan 30-in Convertible Stainless Steel Undercabinet Range Hood ; Undercabinet Range Hoods


In [34]:
# sanity check
import os

#test_dir = os.path.join(user_dir, "selected_test_user")
kgc_dir = "/home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/"

test_fns = [
    os.path.join(kgc_dir, "unified_test/arels.test.sim.tsv"),
    os.path.join(kgc_dir, "unified_test/arels.test.compl.tsv"),
    os.path.join(kgc_dir, "unified_test/qrels.test.tsv")
]
train_fns = [
    os.path.join(kgc_dir, "unified_train/a2sp.train.tsv"),
    os.path.join(kgc_dir, "unified_train/a2cp.train.tsv"),
    os.path.join(kgc_dir, "unified_train/max2_qorient_q2p.train.tsv")
]

for test_fn, train_fn in zip(test_fns, train_fns):
    test_qids, train_qids = set(), set()
    with open(test_fn) as fin:
        for line in fin:
            array = line.rstrip().split("\t")
            assert len(array) == 4
            test_qids.add(array[0])
    with open(train_fn) as fin:
        for line in fin:
            array = line.rstrip().split("\t")
            assert len(array) == 4
            train_qids.add(array[0])
    assert len(test_qids & train_qids) == 0
    print(f"test_fn: {test_fn}, size: {len(test_qids):,}")
    print(f"train_fn: {train_fn}, size: {len(train_qids):,}")
    print(75*"=")
print("SUCCESS: test and train qids not overlap.")

test_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_test/arels.test.sim.tsv, size: 8,273
train_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_train/a2sp.train.tsv, size: 199,507
test_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_test/arels.test.compl.tsv, size: 6,720
train_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_train/a2cp.train.tsv, size: 80,042
test_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_test/qrels.test.tsv, size: 7,274
train_fn: /home/jupyter/unity_jointly_rec_and_search/datasets/unified_kgc/unified_train/max2_qorient_q2p.train.tsv, size: 832,482
SUCCESS: test and train qids not overlap.
